In [29]:
import os
import openai
import subprocess
from dotenv import load_dotenv
import json




def execute_python(file_path):
    # Run the Python file
    result = subprocess.run(['python', file_path], capture_output=True, text=True)

    # Check if the execution was successful
    if result.returncode == 0:
        # Get the output from the executed file
        output = result.stdout.strip()
        #print("Output:", output)
    else:
        # Print the error message if the execution failed
        output = result.stderr.strip()
        #print("Error:", error)
    return output


# Function query GPT-4
def get_response_from_gpt4(sys, usr, assist, tmp=0):
    response = openai.ChatCompletion.create(
    model="gpt-4-0613", # The deployment name you chose when you deployed the ChatGPT or GPT-4 model.
    #model="gpt-3.5-turbo",
    temperature = tmp,
    messages=[
        {"role": "system", "content": sys},
        {"role": "user", "content": usr},
        {"role": "assistant", "content": assist}
    ],

    functions = [
            {
                "name": "execute_python",
                "description": "Execute the Python code generated by GPT",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "pythonscript": {
                            "type": "string",
                            "description": "The Python script containinng the computation generated by GPT.",
                        }
                    },
                    "required": ["pythonscript"],
                },
            }
        ]    
    
    )
    #res = (response['choices'][0]['message']['content'])
    return response

#from dotenv import load_dotenv

load_dotenv("C:\Projrcts\Python\MSJupiter\myenv.env")

#os.environ['API_KEY'] = '100'
api_key = os.getenv("API_KEY")
openai.api_key = api_key

sys = "You are an otstanding programmer. You will be able to write a Python script to perform a required computation."
usr = "Analyze the following query. If you need to perform any computation generate a Python script and use it as a parameeter for the function execute_python. The result should be placed in a variable called result."
#qry = "How much is 2 square."
qry = "What was the day of the week corresponding to 06/20/1951."
assist = "query: " + qry

response = get_response_from_gpt4(sys, usr, assist)
response_message = response["choices"][0]["message"]
res = response['choices'][0]['message']['content']
print(res)
function_name = response_message["function_call"]["name"]
print("function_name:", function_name)
function_args = json.loads(response_message["function_call"]["arguments"])
print("function_args:", function_args)

#Write the file with the generated script
text = function_args['pythonscript']
lines = text.split('\n')
# Write the lines to the file
with open('data/add_numbers.py', 'w') as file:
    for line in lines:
        file.write(line + '\n')
    file.write('print(result)' + '\n')
    

#Specify the path to the Python file you want to execute
file_path = 'data/add_numbers.py'
output = execute_python(file_path)
print("Final Output:", output)

None
function_name: execute_python
function_args: {'pythonscript': "import datetime\n\n# Define the date string\ndate_string = '06/20/1951'\n\n# Convert the date string to a datetime object\ndate_object = datetime.datetime.strptime(date_string, '%m/%d/%Y')\n\n# Get the day of the week\nresult = date_object.strftime('%A')"}
Final Output: Wednesday
